## INF1608 - Análise Numérica - 2016.1
## Departamento de Informática - PUC-Rio 
## Prof. Hélio Lopes - lopes@inf.puc-rio.br
## http://www.inf.puc-rio.br/~lopes

$$$$

## Aluno: Carlos Mattoso
## Matrícula: 1210553


## Lista 1

1) Modifique a implementação da decomposição LU para incluir a permutação de linhas, obtendo a decomposição PA=LU.

2) Implemente a função LUsolve que resolve o sistema Ax=b, tendo em mão a deomposição PA=LU.

3) Utilize as implementações dos items 1 e 2 da lista para resolver o sistema:

A = np.matrix([[1.,0.,0.,1.],[-1.,1.,0.,1.],[-1.,-1.,1.,1.],[-1.,-1.,-1.,1.]])
e
b = np.array([1.,1.,2.,0.])

4) Faça uma função que verique se uma matriz A de tamanho nxn é estritamente diagonal dominante:
        Definição: Uma matriz A (nxn) é estritamente diagonal dominante se para toda linha i vale:
                   $$|A_{i,i}| > \sum_{i=1, i\ne j}^n |A_{i,j}|$$
                   
5) Implemente o método de Jacobi para a solução de um sistema de equações lineares. Defina um critério de parada para o seu método, e explique-o. 
        https://pt.wikipedia.org/wiki/M%C3%A9todo_de_Jacobi

6) Implemente o método de Gauss-Seidel para a solução de um sistema de equações lineares. Defina um critério de parada para o seu método, e explique-o.
        https://pt.wikipedia.org/wiki/M%C3%A9todo_de_Gauss-Seidel

7) Teste muito bem todos esses algoritmos!

## Resolução

In [1]:
import numpy as np

### 1
Apenas fiz uma modificação no código disponibilizado no material do curso em que, antes de se executar o procedimento de decomposição LU, produz-se um vetor que equivale a matriz de pivoteamento $P$. Para isto, percorre-se cada coluna $j$ e determina-se em que linha $i$, $i \geqslant j$, encontra-se o maior valor da coluna corrente; caso este valor absoluto não esteja na diagonal, troca-se a linha de índice igual ao da coluna corrente com a linha em que se encontra o valor. Este vetor indica qual das linhas da matriz A, após o pivoteamento, deve ser a primeira, qual deve ser a segunda e assim em diante.

In [2]:
# Produz o vetor de pivoteamento P
def makePivot(A):
    n = len(A)
    
    # Produz o vetor de pivoteamento `P`
    # Este vetor indica em que linha da estrutura `A` está de fato
    # armazenada determinada linha `i`, já que trocas não são feitas
    # na matriz `A`, apenas "indicadas, em `P`
    P = [i for i in range(n)]
    for j in range(n):
        max_id = max(range(j, n), key=lambda i: abs(A[P[i], j]))
        if j != max_id:
            P[j], P[max_id] = P[max_id], P[j]
    return P
    
def LUdecomp(A):
    LU = np.copy(A)
    n = len(A)

    P = makePivot(A)
    # Trocam-se as linhas de LU segundo A
    for i in range(n):
        LU[i,:] = A[P[i],:]
    
    for j in range(n-1):
        for i in range(j+1,n):
            LU[i][j] = LU[i][j]/LU[j][j]
            for k in range(j+1,n):
                LU[i][k] = LU[i][k] - LU[i][j]*LU[j][k]
    return P, LU

# Resultado esperado:
# [[2.0,  4.0,  7.0],
#  [0.5,  1.0,  1.5],
#  [0.5, -1.0, -2.0]]
print("A")
A = np.matrix([[1., 3., 5.], 
               [2., 4., 7.], 
               [1., 1., 0.]])
print(A)

exp_LU = np.matrix([[2.0,  4.0,  7.0],
                    [0.5,  1.0,  1.5],
                    [0.5, -1.0, -2.0]])
P, LU = LUdecomp(A)
print(P)
print(LU)
print("Erro da A: %f" % np.linalg.norm(LU - exp_LU))

# Matriz dada em sala para caso sem pivoteamento. Resultado esperado:
# [[ 3.          1.        ]
# [ 0.33333333  4.66666667]]
print("\nB")
B = np.matrix([[3.0,1.0],
               [1.0,5.0]])
P, LU = LUdecomp(B)
print(P)
print(LU)

exp_LU = np.matrix([[3. , 1.],
                    [0.33333333, 4.66666667]])
print("Erro da B: %f" % np.linalg.norm(LU - exp_LU))

# Matriz retirada de: http://www4.ncsu.edu/~kksivara/ma505/handouts/lu-pivot.pdf
print("\nC")
C = np.matrix([[2, 1, 1, 0],
               [4, 3, 3, 1],
               [8, 7, 9, 5],
               [6, 7, 9, 8]], float)
P, LU = LUdecomp(C)
print(P)
print(LU)

exp_LU = np.matrix([[8., 7., 9.,5.],
                    [3./4., 7./4., 9./4., 17./4.],
                    [0.5, -2./7., -6./7., -2./7.],
                    [0.25, -3./7., 1./3., 2./3.]], float)
print("Erro da C: %f" % np.linalg.norm(LU - exp_LU))

A
[[ 1.  3.  5.]
 [ 2.  4.  7.]
 [ 1.  1.  0.]]
[1, 0, 2]
[[ 2.   4.   7. ]
 [ 0.5  1.   1.5]
 [ 0.5 -1.  -2. ]]
Erro da A: 0.000000

B
[0, 1]
[[ 3.          1.        ]
 [ 0.33333333  4.66666667]]
Erro da B: 0.000000

C
[2, 3, 1, 0]
[[ 8.          7.          9.          5.        ]
 [ 0.75        1.75        2.25        4.25      ]
 [ 0.5        -0.28571429 -0.85714286 -0.28571429]
 [ 0.25       -0.42857143  0.33333333  0.66666667]]
Erro da C: 0.000000


## 2

Utilizam-se os métodos de `LUforwardsub` e `LUbackwardsub` disponibilizados em aula. Como agora temos um vetor de pivotamento $P$, é necessário primeiramente aplicá-lo a $b$, observe:
$$Ax = b$$
$$PAx = Pb$$
$$LUx = Pb$$
Tendo-se calculado $Pb$, pode-se aplicar o método `LUsolve` disponibilizado no material. Modifico este método para fazer o "cálculo" de $Pb$ dentro dele:

In [3]:
def LUforwardsub (L,b):
    n = len(L)
    y = np.zeros(n)
    y[0] = b[0]
    for i in range(1,n):
        y[i] = b[i]
        for j in range(i):
            y[i] = y[i] - L[i][j]*y[j]
    return y
            
def LUbackwardsub (U,y):
    n = len(U)
    x = np.zeros(n)
    x[n-1] = y[n-1]/U[n-1,n-1]
    for i in range(n-2,-1,-1):
        x[i] = y[i]
        for j in range(i+1,n):
            x[i] = x[i] - U[i][j]*x[j]
        x[i] /= U[i][i]
    return x
    
def LUsolve(P,LU,b):
    n = len(P)
    _b = np.copy(b)
    for i in range(n):
       b[i] = _b[P[i]] 
    
    y = LUforwardsub(LU,b)
    x = LUbackwardsub(LU,y)
    return x

## 3

Resolvamos o sistema para o exemplo acima e comparemos o resultado ao obtido pelo método `solve` de `numpy`:

In [4]:
A = np.matrix([[ 1., 0., 0., 1.],
               [-1., 1., 0., 1.],
               [-1.,-1., 1., 1.],
               [-1.,-1.,-1., 1.]])
b = np.array([1.,1.,2.,0.])

P, LU = LUdecomp(A)
print(P)
print(LU)
x = LUsolve(P,LU,b)
print(x)

x_exp = np.linalg.solve(A,b)
print(x_exp)

print("Erro: %f" % np.linalg.norm(x-x_exp))

[0, 1, 2, 3]
[[ 1.  0.  0.  1.]
 [-1.  1.  0.  2.]
 [-1. -1.  1.  4.]
 [-1. -1. -1.  8.]]
[ 0.  0.  1.  1.]
[ 0.  0.  1.  1.]
Erro: 0.000000


## 4

In [5]:
def isDiagDom(A):
    n = len(A)

    # obtém uma matriz apenas com os elementos da diagonal e o restante zerado
    diag = np.multiply(np.eye(n), A) 
    
    # soma os valores absolutos dos elementos de cada linha, excluindo o da diagonal
    others_sum = np.absolute(A - diag).sum(axis=1) 
    
    # certifica-se de que em todos os casos o elemento da diagonal é maior em módulo
    print((np.absolute(diag.sum(axis=1)) > others_sum).sum() == n) 

diagDom = np.matrix([[10,2,3,4],
                     [1,-6,0,4],
                     [1,1,3,0],
                     [1,2,3,-8]], float)
nonDiagDom = np.matrix([[10,2,3,4],
                        [1,5,0,4],
                        [1,1,3,0],
                        [1,2,3,-8]], float)
isDiagDom(diagDom) # Verdadeiro
isDiagDom(nonDiagDom) # Falso
isDiagDom(np.eye(3)) # I -> Verdadeiro

True
False
True


## 5

O critério de parada principal selecionado foi de que a norma da diferença entre os vetores solução $x^{k+1}$ (nova solução) e $x^{k}$ (solução anterior) fique abaixo de um valor de tolerância `eps` definido pelo usuário. Além disso, pode-se passar um limite de iterações `maxiter` para assegurar que a função irá eventualmente terminar, caso o valor de tolerância jamais seja atingido. Os valores de saída deste método ficam praticamente idênticos ao esperado.

In [6]:
# Implementado com base no pseudo-código da Wikipédia: 
#   https://en.wikipedia.org/wiki/Jacobi_method#Algorithm
def jacobi(A, b, x0, eps, maxiter):
    n = len(A)
    x = np.copy(x0)

    for itr in range(maxiter):
        for i in range(n):
            s = 0.;
            for j in range(n):
                if i != j:
                    s += A[i,j] * x0[j]
            x[i] = (b[i] - s)/A[i,i]
        if np.linalg.norm(x - x0) < eps: 
            break
        x0 = np.copy(x) # x0 sempre tem o valor da iteração anterior
    return x

# Exemplo da Wikipédia: https://en.wikipedia.org/wiki/Jacobi_method#Example
A = np.matrix([[10., -1., 2., 0.],
              [-1., 11., -1., 3.],
              [2., -1., 10., -1.],
              [0.0, 3., -1., 8.]])
b = np.array([6., 25., -11., 15.])

x = jacobi(A,b,np.zeros_like(b), 1e-10, 1000)

print("Solução: ")
print(x)

error = np.dot(A,x) - b
print("Erro:")
print(error)

# Outro exemplo:
A = np.matrix([[2.,1.],
               [5.,7.]])
b = np.array([11.,13.])

x = jacobi(A,b,np.ones_like(b), 1e-10, 1000)

print("Solução: ")
print(x)

error = np.dot(A,x) - b
print("Erro:")
print(error)

Solução: 
[ 1.  2. -1.  1.]
Erro:
[[  7.19939663e-11  -1.32647671e-10   9.33333411e-11  -1.06602727e-10]]
Solução: 
[ 7.11111111 -3.22222222]
Erro:
[[  2.65032440e-12   3.70881992e-10]]


## 6

Utilizou-se o mesmo critério de parada que anteriormente. Aqui utiliza-se apenas o $x$ ao longo do algoritmo para as etapas de computação, já que os valores computados da solução podem ser sobreescritos. Contudo, o $x0$ é mantido para se verificar que o critério de parada foi atingido.

In [8]:
# Implementado com base no pseudo-código da Wikipédia: 
#   https://en.wikipedia.org/wiki/Gauss–Seidel_method#Algorithm
def gauss_seidel(A, b, x0, eps, maxiter):
    n = len(A)
    x = np.copy(x0)
    
    for itr in range(maxiter):
        for i in range(n):
            s = 0.;
            for j in range(n):
                if i != j:
                    s += A[i,j] * x[j] # aqui mudou de x0 para x
            x[i] = (b[i] - s)/A[i,i]
        if np.linalg.norm(x - x0) < eps:
            break
        x0 = np.copy(x) # ainda preserva-se o x da iteração anterior para checar o critério de parada
    return x

# Exemplo da Wikipédia: https://en.wikipedia.org/wiki/Jacobi_method#Example
A = np.matrix([[10., -1., 2., 0.],
              [-1., 11., -1., 3.],
              [2., -1., 10., -1.],
              [0.0, 3., -1., 8.]])
b = np.array([6., 25., -11., 15.])

x = gauss_seidel(A,b,np.zeros_like(b), 1e-10, 1000)

print("Solução: ")
print(x)

error = np.dot(A,x) - b
print("Erro:")
print(error)

# Outro exemplo:
A = np.matrix([[2.,1.],
               [5.,7.]])
b = np.array([11.,13.])

x = gauss_seidel(A,b,np.ones_like(b), 1e-10, 1000)

print("Solução: ")
print(x)

error = np.dot(A,x) - b
print("Erro:")
print(error)

Solução: 
[ 1.  2. -1.  1.]
Erro:
[[ -2.68229883e-11  -1.08677511e-11   5.86197757e-12   0.00000000e+00]]
Solução: 
[ 7.11111111 -3.22222222]
Erro:
[[ -5.03330710e-11   0.00000000e+00]]
